# chapter17 小規模な部品のコンポーザビリティ

## デコレータ
他の関数をラップして、その関数を実行する前後に実行しなければならない振る舞いを定義できる関数

In [1]:
import functools
from collections.abc import Callable
import datetime

In [2]:
def repeat(times: int=1) -> Callable:
    def _repeat(func: Callable):
        def _wrapper(*args, **kwargs):
            for _ in range(times):
                func(*args, **kwargs)
        return _wrapper
    return _repeat

@repeat(times=3)
def say_hello():
    print("hello")

say_hello()

hello
hello
hello


In [3]:
def my_logger(func: Callable):
    def _wrapper(*args, **kwargs):
        print(f"{func.__name__}の実行")
        print(f'開始: {datetime.datetime.now()}')
        ans = func(*args, **kwargs)

        print(f'終了: {datetime.datetime.now()}')
        print(f'実行結果: {ans}')

        return ans
    return _wrapper

@my_logger
def twice(src: int = 5):
    return src*2

twice()

twiceの実行
開始: 2023-11-05 23:11:51.770213
終了: 2023-11-05 23:11:51.770231
実行結果: 10


10

## backoff

In [4]:
import backoff

@backoff.on_exception(backoff.expo,
                      ValueError,
                      max_tries=3)
def test():
    number = int(input("Enter an integer: "))

test()

Enter an integer:  o
Enter an integer:  j
Enter an integer:  o


ValueError: invalid literal for int() with base 10: 'o'

In [ ]:
# db接続とのエラーを検知するときに使用するとよい
import backoff
import requests
from autokitchen.database import OperationException 
# setting properties of self.*_db objects will
# update data in the database

@backoff.on_exception(backoff.expo,
                      OperationException,
                      max_tries=5) 
def on_dish_ordered(dish: Dish):
    self.dish_db[dish].count += 1

@backoff.on_exception(backoff.expo,
                      OperationException,
                      max_tries=5) 
@backoff.on_exception(backoff.expo,
                      requests.exceptions.HTTPError,
                      max_time=60)
def save_inventory_counts(inventory):
    for ingredient in inventory: 
        self.inventory_db[ingredient.name] = ingredient.count



### コンポーザブルなアルゴリズム

In [ ]:
import itertools

Meal = str

@dataclass
class RecommendationPolicy:
    meals: list[str]
    initial_sorting_criteria: Callable
    grouping_criteria: Callable
    secondary_sorting_criteria: Callable
    selection_criteria: Callable
    desired_number_of_recommendations: int

def recommend_meal(policy: RecommendationPolicy) -> list[Meal]:
    meals = policy.meals
    sorted_meals = sorted(meals, key=policy.initial_sorting_criteria, reverse=True)
    grouped_meals = itertools.groupby(sorted_meals, key=policy.grouping_criteria)
    _, top_grouped = next(grouped_meals)
    secondary_sorted = sorted(top_grouped, key=policy.secondary_sorting_criteria, reverse=True)
    candidates = itertools.takewhile(policy.selection_criteria, secondary_sorted)
    return list(candidates)[:policy.desired_number_of_recommendations]


# dummy functions to get code to run
def get_specials():
    return ["abc", "d", "efghi", "jk", "l", "mno", "p"]

def get_proximity_to_surplus_ingredients(meal):
    return len(meal)

get_proximity_to_last_meal = get_proximity_to_surplus_ingredients

def proximity_greater_than_75_percent(meal):
    return len(meal) > .75


meal = recommend_meal(RecommendationPolicy(
    meals=get_specials(),
    initial_sorting_criteria=get_proximity_to_surplus_ingredients,
    grouping_criteria=get_proximity_to_surplus_ingredients,
    secondary_sorting_criteria=get_proximity_to_last_meal,
    selection_criteria=proximity_greater_than_75_percent,
    desired_number_of_recommendations=3)
)

assert meal == ["efghi"]